## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [196]:
from gensim.models import Word2Vec, KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import judicial_splitter
import collections
from pymystem3 import Mystem
mystem = Mystem()
import numpy as np
from tqdm import tqdm_notebook as tqdm
import os
import pandas as pd
from gensim.test.utils import get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import operator
import re
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
import math
from model import Model
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [2]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [3]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [4]:
qa_corpus[0]

['\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае будет решаться вопрос с видом на жительство в Ростове? Не получится ли ,что приобретя квартиру,он не успеет в ней прописаться до окончании срока временной регистрации. С уважением Людмила Евгеньевна.\n',
 'Добрый вечер!Из Вашего вопроса вообще ничего

In [5]:
questions, answers = [], []
for qa in qa_corpus:
    question = qa[0]
    answer = qa[1]
    questions.append(question)
    answers.append(answer)

In [6]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    if del_digit:
        input_text = re.sub('[0-9]', '', input_text)
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

# TF.IDF

In [7]:
def compute_tf(text):
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

In [8]:
def compute_idf(word, corpus):
    return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [9]:
def compute_tfidf(corpus):
    documents_list = []
    for text in tqdm(corpus):
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

#corpus = [['pasta', 'la', 'vista', 'baby', 'la', 'vista'], 
#['hasta', 'siempre', 'comandante', 'baby', 'la', 'siempre'], 
#['siempre', 'comandante', 'baby', 'la', 'siempre']]
#compute_tfidf(corpus)

In [10]:
corpus = [preprocessing(q) for q in questions] + [preprocessing(a) for a in answers]
tfidf = compute_tfidf(corpus)

A Jupyter Widget

In [11]:
tfidf_questions = tfidf[0:len(questions)]
tfidf_answers = tfidf[len(questions):]

# w2v

In [12]:
# если модель без тэгов
model_without_pos = Word2Vec.load('/Users/irene/Downloads/IR/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [13]:
def get_w2v_vectors_paragraph(paragraph, model, tfidf, ind, multiply_tfidf=True, pos=False):
    """Получает вектор для параграфа"""
    lemmas_paragraph = preprocessing(paragraph)
    #print('lemmas_paragraph', lemmas_paragraph)
    if len(lemmas_paragraph) == 0:
        return np.zeros(300)
    else:
        vector_paragraph = []
        for lemma in lemmas_paragraph:
            if pos:
                lemma = lemma + '_' + get_pos(lemma)
            try:
                if multiply_tfidf:
                    vector = model.wv[lemma] * tfidf[ind][lemma]
                else:
                    vector = model.wv[lemma]
            except:
                vector = np.zeros(300)
            vector_paragraph.append(vector)
        vec = np.array(vector_paragraph).sum(axis=0) / len(vector_paragraph)
        return vec.tolist()

In [14]:
def get_w2v_vectors_text(text, model, tfidf, ind, len_par=4, multiply_tfidf=True, pos=False):
    """Получает массив векторов параграфов"""
    paragraphs = judicial_splitter.splitter(text, len_par)
    return [(paragraph, get_w2v_vectors_paragraph(paragraph, model, tfidf, ind, multiply_tfidf=multiply_tfidf, pos=pos)) for paragraph in paragraphs]

In [15]:
def save_w2v_base(answers, model, tfidf, len_par=4, multiply_tfidf=True, pos=False):
    """Индексирует всю базу для поиска через word2vec"""
    id_answer = []
    text_of_paragraph = []
    w2v = []
    for i, answer in tqdm(enumerate(answers)):
        v_paragraphs = get_w2v_vectors_text(answer, model, tfidf, i, len_par=len_par, multiply_tfidf=multiply_tfidf, pos=pos)
        for v_p in v_paragraphs:
            id_answer.append(i)
            text_of_paragraph.append(v_p[0])
            w2v.append(v_p[1])
    return id_answer, text_of_paragraph, w2v

In [16]:
def create_df(name_cols, data_cols):
    df = {}
    for i, name in enumerate(name_cols):
        df[name] = data_cols[i]
    df = pd.DataFrame(data=df)
    return df

# БЕЗ TF.IDF

In [17]:
id_answer, text_of_paragraph, w2v = save_w2v_base(answers, model_without_pos, tfidf_answers, len_par=2, multiply_tfidf=False)

A Jupyter Widget

In [18]:
df = create_df(['id_answer', 'text_of_paragraph', 'w2v'], [id_answer, text_of_paragraph, w2v])

# С TF.IDF

In [19]:
id_answer_tfidf, text_of_paragraph_tfidf, w2v_tfidf = save_w2v_base(answers, model_without_pos, tfidf_answers, len_par=2, multiply_tfidf=True)
df['w2v_tfidf'] = w2v_tfidf

A Jupyter Widget

# С POS-tagging

In [22]:
def get_pos(lemma):
    sentences = model_udpipe.tokenize(lemma)
    for s in sentences:
        model_udpipe.tag(s)
        model_udpipe.parse(s)
    conllu = model_udpipe.write(sentences, "conllu")
    conllu = re.sub('# .+?\n', '', conllu)
    pos = re.search('(.+?\t){3}(.+?)\t', conllu).group(2)
    return pos

In [23]:
# модель с тегами
model_pos = KeyedVectors.load_word2vec_format('/Users/irene/Downloads/tayga_1_2.vec', binary=False)
model_udpipe = Model('russian-ud-2.0-170801.udpipe')
id_answer_pos, text_of_paragraph_pos, w2v_pos = save_w2v_base(answers, model_pos, tfidf_answers, len_par=2, multiply_tfidf=False, pos=True)
df['w2v_pos'] = w2v_pos
id_answer_pos_tfidf, text_of_paragraph_pos_tfidf, w2v_pos_tfidf = save_w2v_base(answers, model_pos, tfidf_answers, len_par=2, pos=True)
df['w2v_pos_tfidf'] = w2v_pos

A Jupyter Widget

A Jupyter Widget

# train my model

https://drive.google.com/file/d/14wqRshysIn0Cs4dDM_WaW4jPTtix1sgD/view?usp=sharing

In [32]:
train_text = [preprocessing(ans_1) for ans_1 in answers]
path = get_tmpfile("my_word2vec.model")
my_model = Word2Vec(train_text, size=300, window=5, min_count=1, workers=4)
my_model.save("my_word2vec.model")
my_id_answer, my_text_of_paragraph, my_w2v = save_w2v_base(answers, my_model, tfidf_answers, len_par=2, multiply_tfidf=False)
df['my_w2v'] = my_w2v

A Jupyter Widget

# d2v

In [26]:
def train_doc2vec(data, epochs=100):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, 
                min_alpha=0.025, epochs=epochs, workers=4, dm=1)

    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [27]:
fname = get_tmpfile("model_doc2vec_QA")
#model_doc2vec = train_doc2vec(df['text_of_paragraph'], epochs=1000)
#model_doc2vec.save(fname)
model_doc2vec = Doc2Vec.load(fname)

In [28]:
def get_d2v_vectors(paragraph, model_doc2vec, steps=5, alpha=0.1):
    """Получает вектор параграфа"""
    lemmas_paragraph = preprocessing(paragraph, del_stopwords=False)
    model_doc2vec.random.seed(100)
    vector = model_doc2vec.infer_vector(lemmas_paragraph, steps=steps, alpha=alpha)
    return vector.tolist()

def save_d2v_base(paragraphs, model_doc2vec, steps=5, alpha=0.1):
    """Индексирует всю базу для поиска через doc2vec"""
    vectors_d2v = []
    for par in tqdm(paragraphs):
        vectors_d2v.append(get_d2v_vectors(par, model_doc2vec, steps=steps, alpha=alpha))
    return vectors_d2v

In [29]:
df['d2v_simple'] = save_d2v_base(df['text_of_paragraph'], model_doc2vec)
df['d2v_hypo'] = save_d2v_base(df['text_of_paragraph'], model_doc2vec, steps=10, alpha=0.025)

A Jupyter Widget

A Jupyter Widget

In [33]:
df.head()

,id_answer,text_of_paragraph,w2v,w2v_tfidf,w2v_pos,w2v_pos_tfidf,mu_w2v,d2v_simple,d2v_hypo,my_w2v
0,0,Добрый вечер!Из Вашего вопроса вообще ничего н...,"[-0.0037454424891620874, 0.0019313085358589888...","[0.0001088905191863887, -4.5868677261751145e-0...","[0.00604688091889808, 0.005329290515204009, -0...","[0.00604688091889808, 0.005329290515204009, -0...","[0.07501664757728577, -0.09495332092046738, -0...","[0.4394567906856537, 1.2520071268081665, -0.14...","[0.17822310328483582, 1.4375102519989014, -0.2...","[0.07204142957925797, -0.07892116904258728, -0..."
1,1,"Оксана, Вы вправе не платить налог, если являе...","[0.043672472070044786, -0.0023205415590813287,...","[0.0020447711533027983, 0.0001374839770218578,...","[-0.01731853481569009, 0.008737271714428636, 0...","[-0.01731853481569009, 0.008737271714428636, 0...","[0.17264589667320251, -0.11381981521844864, -0...","[1.8779473304748535, 1.2621811628341675, -0.29...","[0.8579018712043762, 0.8938591480255127, 0.175...","[0.17516256868839264, -0.07227104902267456, -0..."
2,2,"Здравствуйте, Илья! Можно ли подать приложения...","[0.00849547550897114, -0.008755609241779894, 0...","[0.00019125750559396694, -0.000272295524700894...","[-0.021240784818655812, 0.0026774465921334924,...","[-0.021240784818655812, 0.0026774465921334924,...","[0.16439856588840485, -0.15668083727359772, -0...","[0.26170143485069275, 0.6218246817588806, 0.03...","[-0.40875673294067383, 0.768581211566925, 0.53...","[0.16471920907497406, -0.11509190499782562, -0..."
3,2,"Лица, участвующие в деле, вправе представлять ...","[0.014957115054130554, -0.01150572020560503, -...","[0.00037368200719356537, -0.000752608524635434...","[-0.030454383121634072, 0.0045334138568829405,...","[-0.030454383121634072, 0.0045334138568829405,...","[0.17913705110549927, -0.1371319741010666, -0....","[-0.1003810465335846, 0.22161240875720978, -0....","[-0.38867640495300293, 0.49730849266052246, 0....","[0.1851647049188614, -0.0843329206109047, -0.3..."
4,2,"Такие документы выполняются в форме, установле...","[0.014798891730606556, -0.006705345120280981, ...","[0.0007809624657966197, -0.0004444418009370565...","[-0.049720443636178974, 0.015440374696627259, ...","[-0.049720443636178974, 0.015440374696627259, ...","[0.2423543483018875, -0.08055200427770615, -0....","[0.7417150735855103, -1.196213722229004, 1.634...","[0.43548583984375, -0.8425877690315247, 1.3399...","[0.2474016547203064, -0.012366699054837227, -0..."


In [34]:
df.to_csv('QA_df.csv')

https://drive.google.com/file/d/1CHnCTjIXWjAEmGAPW0_TCH47tm2SgASO/view?usp=sharing

# w2v + d2v (общие функции для поиска)

In [35]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [36]:
def res_v(vectors, names_doc, v_quary):
    res = []
    for i, vector in enumerate(vectors):
        cos_sim = similarity(v_quary, vector)
        res.append((names_doc[i], cos_sim))
    res.sort(key=operator.itemgetter(1), reverse=True)
    return res

In [37]:
def search_w2v(quary, model, vectors_w2v, names_doc, tfidf, ind, multiply_tfidf=True, pos=False, top=5):
    v_quary = get_w2v_vectors_paragraph(quary, model, tfidf, ind, multiply_tfidf=multiply_tfidf, pos=pos)
    res = res_v(vectors_w2v, names_doc, v_quary)[0:top]
    return res

def search_d2v(quary, model, vectors_d2v, names_doc, steps=5, alpha=0.1, top=5):
    v_quary = get_d2v_vectors(quary, model, steps=steps, alpha=alpha)
    res = res_v(vectors_d2v, names_doc, v_quary)[0:top]
    return res

# Обратный индекс

In [38]:
lemmatized_texts = []
for each_f in tqdm(answers):
    lemmatized = ' '.join([x for x in preprocessing(each_f) if x != ' '])
    lemmatized_texts.append(lemmatized)

A Jupyter Widget

In [39]:
vec = CountVectorizer()
X = vec.fit_transform(lemmatized_texts)
df_index = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
words = list(vec.get_feature_names())

In [40]:
df_index.head()

,00,01,02,03,04,05,06,07,08,09,...,январь,январялюдмилая,ярлык,ярослав,ярославский,ясно,ясный,яхта,ячейка,ящик
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
def inverted_index(df) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    files = []
    for word in df:
        sub = []
        docs = np.where(df[word] > 0)[0]
        for f in docs:
            dl = len(lemmatized_texts[f].split())
            fr = round(df[word][f]/dl, 4)
            sub.append([f, dl, fr])
        files.append(sub)
    index = pd.DataFrame(data={'Слово': words, 'Информация': files})
    return index

In [44]:
index = inverted_index(df_index)

In [45]:
index.head()

,Информация,Слово
0,"[[928, 157, 0.0255], [1053, 383, 0.0209], [112...",00
1,"[[28, 72, 0.0139], [47, 39, 0.0513], [68, 228,...",01
2,"[[50, 141, 0.0071], [134, 127, 0.0236], [137, ...",02
3,"[[45, 128, 0.0078], [53, 97, 0.0103], [94, 53,...",03
4,"[[27, 90, 0.0111], [30, 73, 0.0137], [45, 128,...",04


In [46]:
from math import log

k1 = 2.0
b = 0.75
avgdl = round(sum([len(q.split(' ')) for q in lemmatized_texts])/len(lemmatized_texts))#средняя длина док-ов в коллекции
N = len(lemmatized_texts)

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = math.log((N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

In [47]:
def compute_sim(lemma, inverted_index) -> float:
    """
    Compute similarity score between word in search query and all document  from collection
    :return: score
    """
    doc_list = list(inverted_index.loc[inverted_index['Слово'] == lemma]['Информация'])[0]
    relevance_dict = {}
    for doc in doc_list:
        relevance_dict[doc[0]] = score_BM25(doc[2], doc[1], avgdl, k1, b, N, len(doc_list))
    return relevance_dict

In [48]:
def get_search_result(query, top=5) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    query = [que for que in preprocessing(query) if que in words]
    #print(query)
    res = {}
    for word in query:
        relevance_dict = compute_sim(word, index)
        res = {k: res.get(k, 0) + relevance_dict.get(k, 0) for k in set(res) | set(relevance_dict)}
    return sorted(res.items(), key=operator.itemgetter(1), reverse=True)[0:top]

# Blending

## * w2v + d2v

In [177]:
def blend_d2v_w2v(res_w2v, res_d2v, v, top=5):
    res_w2v = [res1 + (k,) for k, res1 in enumerate(res_w2v)]
    res_d2v = [res2 + (j,) for j, res2 in enumerate(res_d2v)]
    res_w2v = sorted(res_w2v, key = lambda x: (x[0], x[2]))
    res_d2v = sorted(res_d2v, key = lambda x: (x[0], x[2]))
    ranges = []
    for i, res3 in enumerate(res_w2v):
        new_range = res3[1] * v + res_d2v[i][1] * (1-v)
        ranges.append((res3[0], new_range))
    return sorted(ranges, key = lambda x: (x[1]), reverse=True)[0:top]

In [178]:
res_w2v = [(1, 0.3), (2, 0.2), (2, 0.1), (3, 0.1), (4, 0.1)]
res_d2v = [(2, 0.8), (1, 0.7), (4, 0.2), (3, 0.1), (2, 0.1)]

In [179]:
blend_d2v_w2v(res_w2v, res_d2v, 0.8, top=5)

[(1, 0.37999999999999995),
 (2, 0.32),
 (4, 0.12000000000000001),
 (2, 0.1),
 (3, 0.1)]

## * w2v + d2v +inverted index

In [ ]:
# Это подумаю с авито

# Расчет точности

In [51]:
def accuracy_index(questions, top=5):
    true = 0
    for i, q in tqdm(enumerate(questions)):
        top5 = [res[0] for res in get_search_result(q, top=top)]
        if i in top5:
            true += 1
    ACCURACY = true / len(questions)
    return ACCURACY

In [52]:
def accuracy_w2v(questions, tfidf_questions, w2v, multiply_tfidf=True, pos=False, top=5):
    true = 0
    for i, q in tqdm(enumerate(questions)):
        top5 = [res[0] for res in search_w2v(q, model_without_pos, w2v, df['id_answer'], tfidf_questions, i, multiply_tfidf=multiply_tfidf, pos=pos, top=top)]
        if i in top5:
            true += 1
    ACCURACY = true / len(questions)
    return ACCURACY

In [61]:
def accuracy_d2v(questions, model, d2v, steps=5, alpha=0.1, top=5):
    true = 0
    for i, q in tqdm(enumerate(questions)):
        top5 = [res[0] for res in search_d2v(q, model_doc2vec, d2v, df['id_answer'], steps=steps, alpha=alpha, top=top)]
        if i in top5:
            true += 1
    ACCURACY = true / len(questions)
    return ACCURACY

### Обратный индекс

In [54]:
acc_5_ind = accuracy_index(questions)
acc_10_ind = accuracy_index(questions, top=10)

A Jupyter Widget

A Jupyter Widget

In [55]:
print('Точность метода "обратный индекс" на top-5:', acc_5_ind)
print('Точность метода "обратный индекс" на top-10:', acc_10_ind)

Точность метода "обратный индекс" на top-5: 0.3092485549132948
Точность метода "обратный индекс" на top-10: 0.3901734104046243


### w2v

In [56]:
acc_5_w2v = accuracy_w2v(questions, tfidf_questions, df['w2v'], multiply_tfidf=False, pos=False)
acc_10_w2v = accuracy_w2v(questions, tfidf_questions, df['w2v'], multiply_tfidf=False, pos=False, top=10)
my_acc_5_w2v = accuracy_w2v(questions, tfidf_questions, df['mu_w2v'], multiply_tfidf=False, pos=False)
my_acc_10_w2v = accuracy_w2v(questions, tfidf_questions, df['mu_w2v'], multiply_tfidf=False, pos=False, top=10)
acc_5_w2v_pos = accuracy_w2v(questions, tfidf_questions, df['w2v_pos'], multiply_tfidf=False, pos=True)
acc_10_w2v_pos = accuracy_w2v(questions, tfidf_questions, df['w2v_pos'], multiply_tfidf=False, pos=True, top=10)
acc_5_w2v_tfidf = accuracy_w2v(questions, tfidf_questions, df['w2v_tfidf'], multiply_tfidf=True, pos=False)
acc_10_w2v_tfidf = accuracy_w2v(questions, tfidf_questions, df['w2v_tfidf'], multiply_tfidf=True, pos=False, top=10)
acc_5_w2v_pos_tfidf = accuracy_w2v(questions, tfidf_questions, df['w2v_pos_tfidf'], multiply_tfidf=True, pos=True)
acc_10_w2v_pos_tfidf = accuracy_w2v(questions, tfidf_questions, df['w2v_pos_tfidf'], multiply_tfidf=True, pos=True, top=10)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [57]:
print('Точность метода "w2v" на top-5:', acc_5_w2v)
print('Точность метода "w2v" на top-10:', acc_5_w2v)
print('Точность метода "my w2v" на top-5:', my_acc_5_w2v)
print('Точность метода "my w2v" на top-10:', my_acc_10_w2v)
print('Точность метода "w2v+pos" на top-5:', acc_5_w2v_pos)
print('Точность метода "w2v+pos" на top-10:', acc_5_w2v_pos)
print('Точность метода "w2v+tfidf" на top-5:', acc_5_w2v_tfidf)
print('Точность метода "w2v+tfidf" на top-10:', acc_5_w2v_tfidf)
print('Точность метода "w2v+pos+tfidf" на top-5:', acc_5_w2v_pos_tfidf)
print('Точность метода "w2v+pos+tfidf" на top-10:', acc_5_w2v_pos_tfidf)

Точность метода "w2v" на top-5: 0.2810693641618497
Точность метода "w2v" на top-10: 0.2810693641618497
Точность метода "my w2v" на top-5: 0.004335260115606936
Точность метода "my w2v" на top-10: 0.008670520231213872
Точность метода "w2v+pos" на top-5: 0.002890173410404624
Точность метода "w2v+pos" на top-10: 0.002890173410404624
Точность метода "w2v+tfidf" на top-5: 0.3041907514450867
Точность метода "w2v+tfidf" на top-10: 0.3041907514450867
Точность метода "w2v+pos+tfidf" на top-5: 0.002167630057803468
Точность метода "w2v+pos+tfidf" на top-10: 0.002167630057803468


### Лучший метод - "w2v+tfidf"

### d2v

In [ ]:
acc_5_d2v = accuracy_d2v(questions, model_doc2vec, df['d2v_simple'])
acc_10_d2v = accuracy_d2v(questions, model_doc2vec, df['d2v_simple'], top=10)
acc_5_d2v_hypo = accuracy_d2v(questions, model_doc2vec, df['d2v_hypo'], steps=10, alpha=0.025)
acc_10_d2v_hypo = accuracy_d2v(questions, model_doc2vec, df['d2v_hypo'], steps=10, alpha=0.025, top=10)

In [68]:
print('Точность метода "d2v" на top-5:', acc_5_d2v)
print('Точность метода "d2v" на top-10:', acc_10_d2v)
print('Точность метода "d2v+гиперпараметры" на top-5:', acc_5_d2v_hypo)
print('Точность метода "d2v+гиперпараметры" на top-10:', acc_10_d2v_hypo)

Точность метода "d2v" на top-5: 0.13800578034682082
Точность метода "d2v" на top-10: 0.16184971098265896
Точность метода "d2v+гиперпараметры" на top-5: 0.1611271676300578
Точность метода "d2v+гиперпараметры" на top-10: 0.18786127167630057


### Лучший метод - "d2v+гиперпараметры"

### Возьмем лучшие модели w2v и d2v

In [197]:
def accuracy_blending(v, questions, tfidf_questions, w2v, d2v, multiply_tfidf=True, pos=False, top=1384, steps=5, alpha=0.1):
    true = 0
    for i, q in enumerate(tqdm(questions)):
        top_w2v = [(res1[0], res1[1]) for res1 in search_w2v(q, model_without_pos, w2v, df['id_answer'], tfidf_questions, i, multiply_tfidf=multiply_tfidf, pos=pos, top=1384)]
        top_d2v = [(res2[0], res2[1]) for res2 in search_d2v(q, model_doc2vec, d2v, df['id_answer'], steps=steps, alpha=alpha, top=1384)]
        top = [res3[0] for res3 in blend_d2v_w2v(top_w2v, top_d2v, v, top=5)]
        if i in top:
            true += 1
    ACCURACY = true / len(questions)
    return ACCURACY

In [198]:
for v in np.linspace(0.1, 0.9, 10):
    acc_blend = accuracy_blending(v, questions, tfidf_questions, df['w2v_tfidf'], df['d2v_hypo'], multiply_tfidf=True, pos=False, top=5, steps=5, alpha=0.1)
    print('Точность метода "d2v+w2v" на top-5 с весами', str(round(v, 1)), 'и', str(round(1-v, 1)), ':', acc_blend)

A Jupyter Widget

Точность метода "d2v+w2v" на top-5 с весами 0.1 и 0.9 : 0.022398843930635837


A Jupyter Widget

Точность метода "d2v+w2v" на top-5 с весами 0.2 и 0.8 : 0.04407514450867052


A Jupyter Widget

...

Точность метода "d2v+w2v" на top-5 с весами 0.7 и 0.3 : 0.2904624277456647


A Jupyter Widget

Точность метода "d2v+w2v" на top-5 с весами 0.8 и 0.2 : 0.3013005780346821


A Jupyter Widget

Точность метода "d2v+w2v" на top-5 с весами 0.9 и 0.1 : 0.3013005780346821
